In [2]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(pheatmap)
library(ggsci)
library(RColorBrewer)
library(viridis)
library(reshape2)
library(ggpubr)
library(tidyverse)

source('/home/jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

# BRI

In [5]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA_meta_data-2024-05-09.csv")
meta_data_subset<-meta_data  %>% filter (sample.visitName%in% c('Flu Year 1 Day 0')) 


In [6]:
aggregated_count_file_list<-paste0("/home/jupyter/BRI_Figures/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_subset$pbmc_sample_id,".csv")
df_list<-mclapply(aggregated_count_file_list,function(x){
df<-read.csv(x,check.names = FALSE, row.names = 1)
colnames(df)<-paste0(gsub("^.*/(.*)\\.csv$", "\\1", x),":",colnames(df))
return (df)    
},mc.cores=60)

In [ ]:
#selected_genes<-c("CXCR5",'TIGIT','PDCD1','LGALS3','CCR4',"SLAMF1","GATA3","KLRB1", "ITGA4", "TOX", "CD84","CCR8", "IL4R", "NOTCH1")#TH2 activity
selected_genes<-c("CD40")


df<-filter_genes_and_celltype(df_list, selected_genes, c('Core memory B cell'), longformat = TRUE)


df<-left_join(df,meta_data_subset)

df<-df %>% 
mutate(AgeGroup=case_when(cohort.cohortGuid=="BR1"~"Young", TRUE~'Older'))

df$AgeGroup<-factor(df$AgeGroup,levels=c("Young","Older"))
df_CD40<-df

In [14]:
selected_genes<-c("CD40LG")


df<-filter_genes_and_celltype(df_list, selected_genes, c('CM CD4 T cell'), longformat = TRUE)


df<-left_join(df,meta_data_subset)

df<-df %>% 
mutate(AgeGroup=case_when(cohort.cohortGuid=="BR1"~"Young", TRUE~'Older'))

df$AgeGroup<-factor(df$AgeGroup,levels=c("Young","Older"))
df_CD40LG<-df

Joining with `by = join_by(pbmc_sample_id)`


In [15]:
df<-rbind(df_CD40LG,df_CD40)

In [16]:
write.csv(df,"mean_exp_selected.csv")